# Ingest Cookbook (DataModels).

Following the concepts on the different [means](https://docs.aperturedata.io/HowToGuides/Ingestion/Ingestion#ingestion-based-on-the-data-model) of ingesting the data, we will build an example using the DataModel method in this notebook.

We will use the [Cookbook dataset](https://docs.google.com/spreadsheets/d/1G1HPG3Dxx5W39OD6b74wMHvWupD7N-DLUbV7tD5owx8/edit?usp=sharing) to have the data persisted onto ApertureDB instance.

> #### Note
> The notebook assumes that the SDK has been setup as per the instructions.
>
> Colab one time setup [instructions](/Setup/client/notebooks?notebooks=colab).
>
> Non ephemeral jupyter server one time setup [instructions](/Setup/client/notebooks?notebooks=notebook).
>


## Needed resources.
- ```create_nested_json.py```: The script merges the first 3 sheets on the source into a json file such that there will be a list of Dishes objects, and each Dish may have multiple ingredients, and each ingredient has miscellaneous properties. This ends up as a json file.


In [1]:
# Get the script to generate the data.json
!wget https://github.com/aperture-data/Cookbook/raw/refs/heads/main/scripts/create_nested_json.py

# Run the script to generate the data.json
!python create_nested_json.py

--2024-10-01 16:33:55--  https://github.com/aperture-data/Cookbook/raw/refs/heads/main/scripts/create_nested_json.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aperture-data/Cookbook/refs/heads/main/scripts/create_nested_json.py [following]
--2024-10-01 16:33:55--  https://raw.githubusercontent.com/aperture-data/Cookbook/refs/heads/main/scripts/create_nested_json.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1757 (1.7K) [text/plain]
Saving to: ‘create_nested_json.py’

create_nested_json. 100%[===================>]   1.72K  --.-KB/s    in 0s      

2024-10-01 16:33:56 (18.6 MB/s) - ‘cre

## Data model definitions.

A popular way to define the schema in python is using pydantic, and we shall use the same to create the associations of our Cookbook.


In [2]:
from typing import List, Optional
from aperturedb.DataModels import  ImageDataModel, IdentityDataModel

class Ingredient(IdentityDataModel):
    Name: str
    other_names: Optional[str] = ""
    macronutrient: Optional[str] = ""
    micronutrient: Optional[str] = ""
    subgroup: Optional[str] = ""
    category: Optional[str] = ""

class Dish(ImageDataModel):
    contributor: str
    name: str
    location: str
    caption: str
    recipe_url: str
    cuisine: str
    dish_id: int
    ingredients: List[Ingredient]


## Create objects of these models.

The Objects get provisioned using the json file generated by running the script ```create_nested_json.py```

Example line from the json file:

> Sample record in dishes.json
>```json
>{
>    "dish_id": 1,
>    "url": "https://raw.githubusercontent.com/aperture-data/Cookbook/refs/heads/main/images/001 Large.jpeg",
>    "type": "main dish",
>    "location": "NJ",
>    "cuisine": "Indian",
>    "recipe_url": "https://www.tarladalal.com/rajma-chawal-punjabi-rajma-chawal-4951r",
>    "contributor": "gautam",
>    "caption": "Beans with rice",
>    "name": "rajma chawal",
>    "ingredients": [
>      {
>        "Name": "red kidney beans",
>        "other_names": "rajma",
>        "category": "vegetarian",
>        "subgroup": "legume",
>        "macronutrient": "protein"
>      },
>      {
>        "Name": "rice",
>        "other_names": "chawal",
>        "category": "vegetarian",
>        "subgroup": "grain",
>        "macronutrient": "carbohydrates"
>      }
>    ]
>  }
>```


These objects can be passed to a function called ```generate_add_query``` which takes care of generating the queries that ApertureDB can execute to persist the objects on the DB.

## Generate and execute queries.

In [6]:
from aperturedb.Query import generate_add_query
from aperturedb.CommonLibrary import  execute_query, create_connector
import json
from tqdm.auto import tqdm

with open("dishes.json") as ins:
    client = create_connector()
    dishes = json.load(ins)
    for dish in tqdm(dishes):
        #Create the Dish object, along with it's ingredients
        dish = Dish(**dish)

        # Create a query to be run against the database
        query, blobs, _ = generate_add_query(dish)

        # Execute the query. The client has been setup.
        result, response, output_blobs = execute_query(client, query, blobs)
        if result != 0:
            print(response)
            break


  0%|          | 0/29 [00:00<?, ?it/s]